In [26]:
import torch 
import numpy as np

from nltk.tokenize import sent_tokenize 

from pathlib import Path 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.bpe import BPETokenizer 
from mingpt.utils import set_seed 
set_seed(1234)

from mingpt.model import GPT
from mingpt.trainer import Trainer
from utils import *
import datasets
from SentimentDataset import SentimentDataset
from LanguageModelingDataset import LanguageModelingDataset


cuda==False


In [27]:
# Instantiate the Training Dataset
train_dataset = SentimentDataset(split="train")  # use this for the short corpus

# Instantiate a Validation Dataset (this is only really needed for the fine-tune task, not the LM task)
val_dataset = LanguageModelingDataset(split="validation")

In [28]:
# Print out an example of the data - this is processed more once it reaches lm_collate_fn (above)
x,y = train_dataset[5]
print(train_dataset[5])
print("X: ",train_dataset.tokenizer.decode(x))
print("Y: ",(y))

(tensor([ 5562,   705,    82,  1290,  1165, 15444,   284, 17004,   884, 31194,
         3513,   220]), 0)
X:  that 's far too tragic to merit such superficial treatment 
Y:  0


In [29]:
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model_config.n_classification_class = 2
model = GPT(model_config)
# model.to(trainer.device)

modelsavename= "model_large100K.pt"
model.load_state_dict(torch.load(modelsavename))

number of parameters: 2.52M


<All keys matched successfully>

In [30]:
# Create a Trainer object and set the core hyper-parameters
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 3000  # For small corpus: 3000 iterations is plenty. For large corpus: 100000 iterations is needed
train_config.num_workers = 0
train_config.batch_size = 8    # For small corpus, batch size of 4 is fine.  For large corpus use 16
trainer = Trainer(train_config, model, train_dataset, val_dataset, collate_fn=sen_collate_fn, downstream_finetune=True)
trainer.set_callback('on_batch_end', sen_batch_end_callback)
# Train!
trainer.run()

running on device cpu


TypeError: only integer tensors of a single element can be converted to an index